In [1]:
import sys
sys.path.append("..")
import functools
import pickle

import numpy as np
import torch
import pyequion2

import tmcdiff

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
TK = 298.15
bulk_velocity = 1e0
pipe_diameter = 1e-2
species = ["C", "Ca", "Na", "Cl"]
phases = ["Calcite"]
c_nahco3 = 0.018
c_cacl2 = 0.006
c_nacl = 0.003
c_mgcl2 = 0.003
molal_balance = {'C':c_nahco3,
                 'Ca':c_cacl2,
                 'Na':c_nahco3 + c_nacl,
                 'Cl':2*c_cacl2 + c_nacl}
ngrid = 20
ypmax = 5

In [4]:
pyequion2.constants.GAS_CONSTANT

8.31446261815324

In [5]:
def get_reaction_constant(A, E, TK):
    logkr = np.log(A) - E/8.31446261815324*(1/TK - 1/298.15)
    return np.exp(logkr)

kr = get_reaction_constant(*(52153.84138874875, 86881.05), 298.15)*1e-6
kr, kr/2

(0.05215384138874875, 0.026076920694374374)

In [6]:
solver = tmcdiff.TransportSolver(species)
solver.kreaction = np.array([kr])
solver.set_flow_conditions(TK, bulk_velocity, pipe_diameter)
solver.set_initial_conditions(molal_balance, phases)
solver.build_transport(ngrid, ypmax)
solver.set_initial_guess()

In [7]:
eqsys_front = pyequion2.EquilibriumSystem(species, from_elements=True,
                                           activity_model="DEBYE")
sol, _ = eqsys_front.solve_equilibrium_mixed_balance(TK, molal_balance=molal_balance)
sol.saturation_indexes

{'Aragonite': 1.4501980854042777,
 'Calcite': 1.593967932796513,
 'Halite': -5.246831827080137,
 'Vaterite': 1.0275364385822137}

In [8]:
solver.solve(100.0, print_frequency=1)

0.006382656748490521%
0.20047855545742288%
0.20311187891243773%
0.20574520236745258%
0.23207843691760113%
0.49541078241908687%
0.5756392676491443%
0.6558677528792017%
0.7414810392054941%
0.8270943255317866%
0.9127076118580788%
1.115463190044745%
1.1541887810139801%
1.1929143719832154%
1.4401443068808282%
1.4844553682959747%
1.528766429711121%
1.5810381795427366%
1.633309929374352%
1.950290487173039%
1.9661395150629735%
1.9819885429529078%
2.1404788218522515%
2.142883729756947%
2.145288637661643%
2.1693377167085988%
2.409828507178159%
2.4329485990310915%
2.456068690884024%
2.687269609413349%
2.74506983904568%
2.802870068678011%
3.101210877487475%
3.1031378791854856%
3.105064880883496%
3.1243348978636005%
3.3170350676646465%
3.355575101624856%
3.394115135585065%
3.779515475187158%
3.8504529177215145%
3.9213903602558715%
4.039338533330755%
4.15728670640564%
4.211516344604284%
4.265745982802929%
4.364114898616259%
4.462483814429588%
4.820297243428648%
4.835112034848824%
4.849926826269%
4.9

52.25491625434185%
52.2849879530107%
52.31505965167953%
52.61577663836797%
52.6308124877024%
52.64584833703682%
52.66088418637124%
52.74024667047522%
53.53387151151506%
53.54180775992545%
53.54974400833585%
53.62910649243983%
54.42273133347967%
54.52285562691268%
54.622979920345685%
55.517568931978246%
55.520952101554684%
55.52433527113112%
55.55816696689547%
55.89648392453899%
55.91740301613066%
55.93832210772234%
56.14751302363916%
56.189351206822536%
56.2311893900059%
56.64957122183955%
56.65538547901001%
56.66119973618048%
56.71934230788509%
57.30076802493124%
57.329839310783534%
57.358910596635845%
57.64962345515892%
57.654455213762546%
57.659286972366175%
57.70760455840245%
58.19078041876527%
58.1956121773689%
58.20044393597252%
58.24876152200882%
58.731937382371626%
58.85273134746233%
58.973525312553036%
59.21380825809194%
59.23153761526735%
59.24926697244277%
59.42656054419691%
59.46201925854774%
59.497477972898565%
59.85206511640685%
59.88752383075768%
59.92298254510851%
59.98

In [10]:
solver.save("data/exp3_0")